In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [45]:
## Load the VGG16 model 
model = tf.keras.applications.VGG16(
    include_top=True, weights='imagenet', input_tensor=None, input_shape=None,
    pooling=None, classes=1000, classifier_activation='softmax'
)


In [46]:
print(model)

In [47]:
## Model inputs
model.inputs

[<tf.Tensor 'input_2:0' shape=(None, 224, 224, 3) dtype=float32>]

In [48]:
## Model outputs
model.outputs

[<tf.Tensor 'predictions/Softmax_1:0' shape=(None, 1000) dtype=float32>]

In [49]:
## Set image height and width for preprocessing 
img_height = 224
img_width = 224

In [50]:
## Import classification labels. See imagenet_classes.py
from imagenet_classes import class_names

In [54]:
## Local path of image to classify
img_path = 'tricycle.jpg'

In [55]:
## Preprocess the image 
img = keras.preprocessing.image.load_img(
    img_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

In [56]:
## Classify the image
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


This image most likely belongs to tricycle, trike, velocipede with a 0.27 percent confidence.


In [57]:
## Extract a layer from the model. 
## Layer names can be found here: https://github.com/tensorflow/tensorflow/blob/v2.3.1/tensorflow/python/keras/applications/vgg16.py
## Guide: https://www.gcptutorials.com/post/how-to-extract-features-from-layers-in-tensorflow 
features_layer1 = tf.keras.models.Model(
    inputs=model.inputs,
    outputs=model.get_layer(name='block1_conv1').output,
)

print(features_layer1)

In [58]:
## Example: Running the input through only this layer: 
features_layer1(img_array)

<tf.Tensor: shape=(1, 224, 224, 64), dtype=float32, numpy=
array([[[[  0.       ,  53.05183  ,   0.       , ...,   0.       ,
           57.581234 ,   0.       ],
         [  0.       ,  49.690525 , 150.26212  , ...,   0.       ,
          591.58234  , 302.26395  ],
         [  0.       ,  49.690525 , 150.26212  , ...,   0.       ,
          591.58234  , 302.26395  ],
         ...,
         [  0.       ,  49.690525 , 150.26212  , ...,   0.       ,
          591.58234  , 302.26395  ],
         [  0.       ,  49.690525 , 150.26212  , ...,   0.       ,
          591.58234  , 302.26395  ],
         [  4.656199 ,  35.0211   , 316.7973   , ...,   0.       ,
          943.8628   , 715.88824  ]],

        [[  0.       ,  68.91937  ,   0.       , ...,   0.       ,
            0.       ,   0.       ],
         [  3.71517  ,  56.749004 , 115.33566  , ...,   0.       ,
            2.9643915,  21.566965 ],
         [  3.71517  ,  56.749004 , 115.33566  , ...,   0.       ,
            2.9643915,  21

In [62]:
## Get weights from this layer. 
features_layer1.weights

[<tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
 array([[[[ 4.29470569e-01,  1.17273867e-01,  3.40129584e-02, ...,
           -1.32241577e-01, -5.33475243e-02,  7.57738389e-03],
          [ 5.50379455e-01,  2.08774377e-02,  9.88311544e-02, ...,
           -8.48205537e-02, -5.11389151e-02,  3.74943428e-02],
          [ 4.80015397e-01, -1.72696680e-01,  3.75577137e-02, ...,
           -1.27135560e-01, -5.02991639e-02,  3.48965675e-02]],
 
         [[ 3.73466998e-01,  1.62062630e-01,  1.70863140e-03, ...,
           -1.48207128e-01, -2.35300660e-01, -6.30356818e-02],
          [ 4.40074533e-01,  4.73412387e-02,  5.13819456e-02, ...,
           -9.88498852e-02, -2.96195745e-01, -7.04357103e-02],
          [ 4.08547401e-01, -1.70375049e-01, -4.96297423e-03, ...,
           -1.22360572e-01, -2.76450396e-01, -3.90796512e-02]],
 
         [[-6.13601133e-02,  1.35693997e-01, -1.15694344e-01, ...,
           -1.40158370e-01, -3.77666801e-01, -3.00509870e-01],
    

In [3]:
# Begin transfer learning 
## Load the VGG16 model 
base_model = tf.keras.applications.VGG16(
    include_top=True, weights='imagenet', input_tensor=None, input_shape=None,
    pooling=None, classes=1000, classifier_activation='softmax'
)

In [4]:
# Freeze the convolutional base
base_model.trainable = False

In [5]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
# Add a classification head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)


NameError: name 'feature_batch' is not defined

In [7]:
# Create the added prediction layer 
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


NameError: name 'feature_batch_average' is not defined

In [ ]:
# Link: https://www.tensorflow.org/tutorials/images/transfer_learning 